In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

# Make sure that caffe is on the python path:
caffe_root = '../' # Expected to be in rank_scripts/

import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

import os

In [5]:
caffe.set_device(0)
caffe.set_mode_gpu()
net = caffe.Net(caffe_root +  'rank_scripts/multi/deploy_multinet.prototxt',
                caffe_root + 'rank_scripts/models103/_iter_500.caffemodel',
                caffe.TEST)
net1 = caffe.Net(caffe_root +  'rank_scripts/multi/deploy_multinet.prototxt',
                caffe_root + 'rank_scripts/models103/_iter_500.caffemodel',
                caffe.TEST)

# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
#transformer.set_mean('data', np.load(caffe_root + 'rank_scripts/query_256x128_market.npy').mean(1).mean(1)) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB

In [7]:
# set net to batch size of 50
net.blobs['data'].reshape(50,3,227,227)
net1.blobs['data'].reshape(50,3,227,227)

In [23]:
net.blobs['data'].data[...] = transformer.preprocess('data', caffe.io.load_image('/home/jayant/vision/PETA/i-LID_alex/archive/0005_002.jpg'))
#check_image =  caffe.io.load_image(caffe_root + 'rank_scripts/images_market/0002_c1s2_050821_02.jpg')
out = net.forward()
vector1=out['fc7'][0]
print vector1
#plt.imshow(check_image)

[-17.52217484  -2.74302793 -25.95532036 ..., -16.93779564 -20.23269272
 -10.90326118]


In [27]:
net1.blobs['data'].data[...] = transformer.preprocess('data', caffe.io.load_image('/home/jayant/vision/PETA/i-LID_alex/archive/0003_001.jpg'))
out1 = net1.forward()
vector2=out1['fc7'][0]
print vector2

[-14.26633739 -18.01190948 -18.61690331 ...,  -9.3441267   -4.8424325
  -6.29477739]


In [28]:
# calculating euclidean distance
sum=0
for i in range(4096):
        diff= vector1[i]-vector2[i]
        diff=diff*diff
        sum=sum+diff
        
print math.sqrt(sum)

747.432931283
